In [1]:
# for gioia's python build
import sys
sys.path.append('/Users/maggiori/anaconda/envs/py35/lib/python3.5/site-packages')

In [2]:
import numpy as np

In [3]:
# to import modules locally without having installed the entire package
# http://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

In [4]:
from tsdb import PersistentDB
from tsdb import BitMapIndex
from timeseries import TimeSeries

In [5]:
# synthetic data
t = np.array([1, 1.5, 2, 2.5, 10, 11, 12])
v1 = np.array([10, 12, -11, 1.5, 10, 13, 17])
v2 = np.array([8, 12, -11, 1.5, 10, 13, 17])
a1 = TimeSeries(t, v1)
a2 = TimeSeries(t, v2)

In [6]:
identity = lambda x: x

# index: 1 is binary tree index, 2 is bitmap index
schema = {
  'pk':         {'type': 'str', 'convert': identity,   'index': None, 'values': None},
  'ts':         {'type': 'int', 'convert': identity,   'index': None, 'values': None},
  'order':      {'type': 'int', 'convert': int,        'index': 1, 'values': None},
  'blarg':      {'type': 'int', 'convert': int,        'index': 1, 'values': None},
  'useless':    {'type': 'int', 'convert': identity,   'index': 1, 'values': None},
  'mean':       {'type': 'float', 'convert': float,    'index': 1, 'values': None},
  'std':        {'type': 'float', 'convert': float,    'index': 1, 'values': None},
  'vp':         {'type': 'bool', 'convert': bool,      'index': 2, 'values': [True, False]},
  'deleted':    {'type': 'bool', 'convert': bool,      'index': 2, 'values': [True, False]}
}

In [7]:
# Delete any default db present (otherwise the db creation will load
# the previous one...)
filelist = ["../db_files/default/" + f for f in os.listdir("../db_files/default/")]
for f in filelist:
    os.remove(f)

In [10]:
# create persistent db
ddb = PersistentDB(schema, 'pk', len(t), data_dir='../db_files')

After init pks:  {}


In [ ]:
ddb.insert_ts('pk1', a1)

In [ ]:
ddb.upsert_meta('pk1', {'order': 1, 'blarg': 2})

In [ ]:
ddb.insert_ts('pk2', a2)

In [ ]:
ddb.upsert_meta('pk2', {'order': 2, 'blarg': 2})

In [ ]:
ddb.indexes

In [ ]:
# print metadata
for field, index in ddb.indexes.items():
    print(field, index.items())

In [ ]:
ddb.pks.keys()

In [ ]:
ddb.pks.keys()

In [ ]:
ddb.select({'pk': 'pk1'}, [], None)

In [ ]:
ddb.select({'pk': 'pk2'}, [], None)

In [ ]:
ddb.delete_ts('pk1')

In [ ]:
ddb.insert_ts('pk1', a1)

In [ ]:
# print metadata
for field, index in ddb.indexes.items():
    if isinstance(index, BitMapIndex):
        print(field)
        for k in index.keys():
            print(k, index[k])
        print('-----')

In [ ]:
ddb.insert_ts('pk3', a1)
ddb.insert_ts('pk4', a1)
ddb.insert_ts('pk5', a1)
ddb.insert_ts('pk6', a1)
ddb.insert_ts('pk7', a1)

In [ ]:
# print metadata
for field, index in ddb.indexes.items():
    print(field)
    for k in index.keys():
        print(k, index[k])
    print('-----')

In [ ]:
ddb.upsert_meta('pk4', {'vp': True})
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

In [ ]:
ddb.upsert_meta('pk5', {'vp': True})
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

In [ ]:
ddb.upsert_meta('pk5', {'vp': False})
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

In [ ]:
ddb.upsert_meta('pk5', {'vp': True})
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

In [ ]:
ddb.delete_ts('pk5')

In [ ]:
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

In [ ]:
ddb.delete_ts('pk4')

In [ ]:
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

In [ ]:
ddb.insert_ts('pk4', a1)
ddb.insert_ts('pk5', a1)
ddb.upsert_meta('pk4', {'vp': True})
ddb.upsert_meta('pk5', {'vp': True})

In [ ]:
ddb.select({'vp': True}, ['vp'], None)

In [ ]:
ddb.select({'vp': False}, ['vp'], None)

In [ ]:
# bitmap index errors??
print(ddb.indexes['vp'].values())
print(ddb.indexes['vp'].items())

In [ ]:
[ddb.indexes['vp'][k] for k in ddb.indexes['vp'].keys()]

In [ ]:
print(ddb.indexes['vp'][True])
print(ddb.indexes['vp'][False])

In [ ]:
ddb = None